In [1]:
!pip install numpy==1.15


     |████████████████████████████████| 13.9MB 1.3MB/s 
ERROR: tensorflow 1.15.0 has requirement numpy<2.0,>=1.16.0, but you'll have numpy 1.15.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement numpy>=1.16.0, but you'll have numpy 1.15.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: astropy 4.0 has requirement numpy>=1.16, but you'll have numpy 1.15.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.17.5
    Uninstalling numpy-1.17.5:
      Successfully uninstalled numpy-1.17.5


In [0]:

!pip install pytorch_transformers
!pip install pytorch-pretrained-bert pytorch-nlp

In [0]:

# Upload the train file from your local drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)


In [0]:
cd /content/drive/My\ Drive/Academics/Project/BERT/Bert-Multi-Label-Text-Classification-master/

In [0]:

#%run  run_bert.py --do_data --do_train --data_name uc --train_batch_size 64 --eval_batch_size 1024
!python  run_bert.py --do_test --do_lower_case --data_name uc  

In [0]:
!ls pybert/output/checkpoints/bert/

 checkpoint-epoch-10   checkpoint-epoch-50	  checkpoint_info.bin
 checkpoint-epoch-17   checkpoint-epoch-60	  config.json
 checkpoint-epoch-20   checkpoint-epoch-70	  __init__.py
 checkpoint-epoch-30   checkpoint-epoch-80	  pytorch_model.bin
 checkpoint-epoch-40  'checkpoint_info (1).bin'   training_args.bin


In [0]:
!ls pybert/pretrain/bert/base-uncased/

bert_vocab.txt	config.json  __init__.py  pytorch_model.bin


In [0]:

!CUDA_LAUNCH_BLOCKING=1 python run_bert.py --do_data --do_train --valid_size 0 --data_name uc --train_batch_size 128 --eval_batch_size  128 --epochs 100 --weight_decay 0.01 --train_max_seq_len 16 --eval_max_seq_len 16 --resume_path pybert/output/checkpoints/bert/checkpoint-epoch-30/

Training/evaluation parameters Namespace(adam_epsilon=1e-08, arch='bert', data_name='uc', do_data=True, do_lower_case=False, do_test=False, do_train=True, epochs=100, eval_batch_size=128, eval_max_seq_len=16, fp16=False, fp16_opt_level='O1', grad_clip=1.0, gradient_accumulation_steps=1, learning_rate=2e-05, local_rank=-1, loss_scale=0, mode='min', monitor='valid_loss', n_gpu='0', resume_path='pybert/output/checkpoints/bert/checkpoint-epoch-30/', save_best=False, seed=42, sorted=1, train_batch_size=128, train_max_seq_len=16, valid_size=0.0, warmup_proportion=0.1, weight_decay=0.01)
NumExpr defaulting to 2 threads.


 Mild pallor +   Physiotherapy   Knee Pains
Burning feet


 SCALY LESIONS OVER BOTH LEGS      C/O WHITE DISCHARGE
C/O ICTHING PERIURETHRAL REGION
C/O ITCHING OVER LEFT LEG


 CLASS 2 EXERTIONAL ANGINA


 INSOMNIA, APD, ALLERGY,


 Poly myalgia


 cpts of discharge pv


 Intermittant chest pain while walking
Generalised weekness of body
Increased frequency of urination


 BAC

In [0]:
# %pycat  pybert/model/nn/bert_for_multi_label.py
# %pycat pybert/train/trainer.py
# %pycat pybert/io/bert_processor
# %pycat run_bert.py
!nvidia-smi
# !top
# %pycat pybert/train/metrics.py

# import torch
# input = torch.randn(5, 3, 10)
# input.unsqueeze(3)

Fri Jan 17 09:17:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
%%writefile  run_bert.py

import torch
import warnings
from pathlib import Path
from argparse import ArgumentParser
from pybert.train.losses import BCEWithLogLoss
from pybert.train.trainer import Trainer
from torch.utils.data import DataLoader
from pybert.io.bert_processor import BertProcessor
from pybert.common.tools import init_logger, logger
from pybert.common.tools import seed_everything
from pybert.configs.basic_config import config
from pybert.model.nn.bert_for_multi_label import BertForMultiLable
from pybert.preprocessing.preprocessor import EnglishPreProcessor
from pybert.callback.modelcheckpoint import ModelCheckpoint
from pybert.callback.trainingmonitor import TrainingMonitor
from pybert.train.metrics import AUC, AccuracyThresh, MultiLabelReport,Recall,Acc
from pytorch_transformers import AdamW, WarmupLinearSchedule
from torch.utils.data import RandomSampler, SequentialSampler

warnings.filterwarnings("ignore")


def run_train(args):
   # --------- data
   processor = BertProcessor(vocab_path=config['bert_vocab_path'], do_lower_case=args.do_lower_case)
   label_list = processor.get_labels()
   label2id = {label: i for i, label in enumerate(label_list)}
   id2label = {i: label for i, label in enumerate(label_list)}

   train_data = processor.get_train(config['data_dir'] / f"{args.data_name}.train.pkl")
   train_examples = processor.create_examples(lines=train_data,
                                              example_type='train',
                                              cached_examples_file=config[
                                                   'data_dir'] / f"cached_train_examples_{args.arch}")
   train_features = processor.create_features(examples=train_examples,
                                              max_seq_len=args.train_max_seq_len,
                                              cached_features_file=config[
                                                   'data_dir'] / "cached_train_features_{}_{}".format(
                                                  args.train_max_seq_len, args.arch
                                              ))
#    print(train_features[0].input_ids)
#    print(train_features[0].segment_ids)
#    print(train_features[0].label_id)
#    print(train_features[0].input_mask)
   train_dataset = processor.create_dataset(train_features, is_sorted=args.sorted)
   if args.sorted:
       train_sampler = SequentialSampler(train_dataset)
   else:
       train_sampler = RandomSampler(train_dataset)
   train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

   valid_data = processor.get_dev(config['data_dir'] / f"{args.data_name}.valid.pkl")
   valid_examples = processor.create_examples(lines=valid_data,
                                              example_type='valid',
                                              cached_examples_file=config[
                                                                       'data_dir'] / f"cached_valid_examples_{args.arch}")

   valid_features = processor.create_features(examples=valid_examples,
                                              max_seq_len=args.eval_max_seq_len,
                                              cached_features_file=config[
                                                                       'data_dir'] / "cached_valid_features_{}_{}".format(
                                                  args.eval_max_seq_len, args.arch
                                              ))
   valid_dataset = processor.create_dataset(valid_features)
   valid_sampler = SequentialSampler(valid_dataset)
   valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=args.eval_batch_size)

   # ------- model
   logger.info("initializing model")
   if args.resume_path:
       args.resume_path = Path(args.resume_path)
       model = BertForMultiLable.from_pretrained(args.resume_path)
   else:
       model = BertForMultiLable.from_pretrained(config['bert_model_dir'])
   t_total = int(len(train_dataloader) / args.gradient_accumulation_steps * args.epochs)

   param_optimizer = list(model.named_parameters())
   no_decay = ['bias', 'LayerNorm.weight']
   optimizer_grouped_parameters = [
       {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],'weight_decay': args.weight_decay},
       {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
   ]
   warmup_steps = int(t_total * args.warmup_proportion)
   optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
   lr_scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_steps, t_total=t_total)

   if args.fp16:
       try:
           from apex import amp
       except ImportError:
           raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
       model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

   # ---- callbacks
   logger.info("initializing callbacks")
   train_monitor = TrainingMonitor(file_dir=config['figure_dir'], arch=args.arch)
   model_checkpoint = ModelCheckpoint(checkpoint_dir=config['checkpoint_dir'],mode=args.mode,
                                      monitor=args.monitor,arch=args.arch,
                                      save_best_only=args.save_best)

   # **************************** training model ***********************
   logger.info("***** Running training *****")
   logger.info("  Num examples = %d", len(train_examples))
   logger.info("  Num Epochs = %d", args.epochs)
   logger.info("  Total train batch size (w. parallel, distributed & accumulation) = %d",
               args.train_batch_size * args.gradient_accumulation_steps * (
                   torch.distributed.get_world_size() if args.local_rank != -1 else 1))
   logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
   logger.info("  Total optimization steps = %d", t_total)

   trainer = Trainer(n_gpu=args.n_gpu,
                     model=model,
                     epochs=args.epochs,
                     logger=logger,
                     criterion=BCEWithLogLoss(reduction='sum'),
                     optimizer=optimizer,
                     lr_scheduler=lr_scheduler,
                     early_stopping=None,
                     training_monitor=train_monitor,
                     fp16=args.fp16,
                     resume_path=args.resume_path,
                     grad_clip=args.grad_clip,
                     model_checkpoint=model_checkpoint,
                     gradient_accumulation_steps=args.gradient_accumulation_steps,
                     batch_metrics=[AccuracyThresh(thresh=0.5),Recall(),Acc()],
                     epoch_metrics=[#AUC(average='micro', task_type='binary'),
                                    #MultiLabelReport(id2label=id2label)
                                    AccuracyThresh(thresh=0.5),
                                    Recall(),Acc()])
   trainer.train(train_data=train_dataloader, valid_data=valid_dataloader, seed=args.seed)


def run_test(args):
   from pybert.io.task_data import TaskData
   from pybert.test.predictor import Predictor
   data = TaskData()
   targets, sentences = data.read_data(raw_data_path=config['test_path'],
                                       preprocessor=EnglishPreProcessor(),
                                       is_train=False)
   lines = list(zip(sentences, targets))
   processor = BertProcessor(vocab_path=config['bert_vocab_path'], do_lower_case=args.do_lower_case)
   label_list = processor.get_labels()
   id2label = {i: label for i, label in enumerate(label_list)}

   test_data = processor.get_test(lines=lines)
   test_examples = processor.create_examples(lines=test_data,
                                             example_type='test',
                                             cached_examples_file=config[
                                                                      'data_dir'] / f"cached_test_examples_{args.arch}")
   test_features = processor.create_features(examples=test_examples,
                                             max_seq_len=args.eval_max_seq_len,
                                             cached_features_file=config[
                                                                      'data_dir'] / "cached_test_features_{}_{}".format(
                                                 args.eval_max_seq_len, args.arch
                                             ))
   test_dataset = processor.create_dataset(test_features)
   test_sampler = SequentialSampler(test_dataset)
   test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=args.train_batch_size)
   model = BertForMultiLable.from_pretrained(config['checkpoint_dir'])

   # ----------- predicting
   logger.info('model predicting....')
   predictor = Predictor(model=model,
                         logger=logger,
                         n_gpu=args.n_gpu)
   result = predictor.predict(data=test_dataloader)
   print(result)


def main():
   parser = ArgumentParser()
   parser.add_argument("--arch", default='bert', type=str)
   parser.add_argument("--do_data", action='store_true')
   parser.add_argument("--do_train", action='store_true')
   parser.add_argument("--do_test", action='store_true')
   parser.add_argument("--save_best", action='store_true')
   parser.add_argument("--do_lower_case", action='store_true')
   parser.add_argument('--data_name', default='kaggle', type=str)
   parser.add_argument("--epochs", default=6, type=int)
   parser.add_argument("--resume_path", default='', type=str)
   parser.add_argument("--mode", default='min', type=str)
   parser.add_argument("--monitor", default='valid_loss', type=str)
   parser.add_argument("--valid_size", default=0.2, type=float)
   parser.add_argument("--local_rank", type=int, default=-1)
   parser.add_argument("--sorted", default=1, type=int, help='1 : True  0:False ')
   parser.add_argument("--n_gpu", type=str, default='0', help='"0,1,.." or "0" or "" ')
   parser.add_argument('--gradient_accumulation_steps', type=int, default=1)
   parser.add_argument("--train_batch_size", default=8, type=int)
   parser.add_argument('--eval_batch_size', default=8, type=int)
   parser.add_argument("--train_max_seq_len", default=256, type=int)
   parser.add_argument("--eval_max_seq_len", default=256, type=int)
   parser.add_argument('--loss_scale', type=float, default=0)
   parser.add_argument("--warmup_proportion", default=0.1, type=int, )
   parser.add_argument("--weight_decay", default=0.01, type=float)
   parser.add_argument("--adam_epsilon", default=1e-8, type=float)
   parser.add_argument("--grad_clip", default=1.0, type=float)
   parser.add_argument("--learning_rate", default=2e-5, type=float)
   parser.add_argument('--seed', type=int, default=42)
   parser.add_argument('--fp16', action='store_true')
   parser.add_argument('--fp16_opt_level', type=str, default='O1')

   args = parser.parse_args()
   config['checkpoint_dir'] = config['checkpoint_dir'] / args.arch
   config['checkpoint_dir'].mkdir(exist_ok=True)
   # Good practice: save your training arguments together with the trained model
   torch.save(args, config['checkpoint_dir'] / 'training_args.bin')
   seed_everything(args.seed)
   init_logger(log_file=config['log_dir'] / f"{args.arch}.log")

   logger.info("Training/evaluation parameters %s", args)

   if args.do_data:
       from pybert.io.task_data import TaskData
       data = TaskData()
       targets, sentences = data.read_data(raw_data_path=config['raw_data_path'],
                                           preprocessor=EnglishPreProcessor(),
                                           is_train=True)
       data.train_val_split(X=sentences, y=targets, shuffle=True, stratify=False,
                            valid_size=args.valid_size, data_dir=config['data_dir'],
                            data_name=args.data_name)
   if args.do_train:
       run_train(args)

   if args.do_test:
       run_test(args)


if __name__ == '__main__':
   main()

Writing run_bert.py


In [0]:

%pycat pybert/output/checkpoints/bert/config.json

In [0]:
{
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522,
  "num_labels":200,
  "layer_norm_eps":1e-12
}

In [0]:
%pycat /content/drive/My Drive/Academics/Project/BERT/Bert-Multi-Label-Text-Classification-master/pybert/io/bert_processor.py

In [0]:
!rm run_bert.py

In [0]:
%%writefile pybert/io/bert_processor.py
import csv
import torch
import numpy as np
from ..common.tools import load_pickle
from ..common.tools import logger
from ..callback.progressbar import ProgressBar
from torch.utils.data import TensorDataset
from pytorch_transformers import BertTokenizer

class InputExample(object):
    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid   = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label  = label

class InputFeature(object):
    '''
    A single set of features of data.
    '''
    def __init__(self,input_ids,input_mask,segment_ids,label_id,input_len):
        self.input_ids   = input_ids
        self.input_mask  = input_mask
        self.segment_ids = segment_ids
        self.label_id    = label_id
        self.input_len = input_len

class BertProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def __init__(self,vocab_path,do_lower_case):
        self.tokenizer = BertTokenizer(vocab_path,do_lower_case)

    def get_train(self, data_file):
        """Gets a collection of `InputExample`s for the train set."""
        return self.read_data(data_file)

    def get_dev(self, data_file):
        """Gets a collection of `InputExample`s for the dev set."""
        return self.read_data(data_file)

    def get_test(self,lines):
        return lines

    def get_labels(self):
        """Gets the list of labels for this data set."""
        return ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

    @classmethod
    def read_data(cls, input_file,quotechar = None):
        """Reads a tab separated value file."""
        if 'pkl' in str(input_file):
            lines = load_pickle(input_file)
        else:
            lines = input_file
        return lines

    def truncate_seq_pair(self,tokens_a,tokens_b,max_length):
        # This is a simple heuristic which will always truncate the longer sequence
        # one token at a time. This makes more sense than truncating an equal percent
        # of tokens from each, since if one sequence is very short then each token
        # that's truncated likely contains more information than a longer sequence.
        while True:
            total_length = len(tokens_a) + len(tokens_b)
            if total_length <= max_length:
                break
            if len(tokens_a) > len(tokens_b):
                tokens_a.pop()
            else:
                tokens_b.pop()

    def create_examples(self,lines,example_type,cached_examples_file):
        '''
        Creates examples for data
        '''
        pbar = ProgressBar(n_total = len(lines))
        if cached_examples_file.exists():
            logger.info("Loading examples from cached file %s", cached_examples_file)
            examples = torch.load(cached_examples_file)
        else:
            examples = []
            for i,line in enumerate(lines):
                guid = '%s-%d'%(example_type,i)
                text_a = line[0]
                label = line[1][0]
                print('label',label[0])
                if isinstance(label,str):
                    label = [np.float(x) for x in label.split(",")]
                else:
                    label = [np.float(x) for x in label]
                text_b = None
                example = InputExample(guid = guid,text_a = text_a,text_b=text_b,label= label)
                examples.append(example)
                pbar.batch_step(step=i,info={},bar_type='create examples')
            logger.info("Saving examples into cached file %s", cached_examples_file)
            torch.save(examples, cached_examples_file)
        return examples

    def create_features(self,examples,max_seq_len,cached_features_file):
        '''
        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids:   0   0   0   0  0     0   0
        '''
        pbar = ProgressBar(n_total=len(examples))
        if cached_features_file.exists():
            logger.info("Loading features from cached file %s", cached_features_file)
            features = torch.load(cached_features_file)
        else:
            features = []
            for ex_id,example in enumerate(examples):
                tokens_a = self.tokenizer.tokenize(example.text_a)
                tokens_b = None
                label_id = example.label

                if example.text_b:
                    tokens_b = self.tokenizer.tokenize(example.text_b)
                    # Modifies `tokens_a` and `tokens_b` in place so that the total
                    # length is less than the specified length.
                    # Account for [CLS], [SEP], [SEP] with "- 3"
                    self.truncate_seq_pair(tokens_a,tokens_b,max_length = max_seq_len - 3)
                else:
                    # Account for [CLS] and [SEP] with '-2'
                    if len(tokens_a) > max_seq_len - 2:
                        tokens_a = tokens_a[:max_seq_len - 2]
                tokens = ['[CLS]'] + tokens_a + ['[SEP]']
                segment_ids = [0] * len(tokens)
                if tokens_b:
                    tokens += tokens_b + ['[SEP]']
                    segment_ids += [1] * (len(tokens_b) + 1)

                input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
                input_mask = [1] * len(input_ids)
                padding = [0] * (max_seq_len - len(input_ids))
                input_len = len(input_ids)

                input_ids   += padding
                input_mask  += padding
                segment_ids += padding

                assert len(input_ids) == max_seq_len
                assert len(input_mask) == max_seq_len
                assert len(segment_ids) == max_seq_len

                if ex_id < 2:
                    logger.info("*** Example ***")
                    logger.info(f"guid: {example.guid}" % ())
                    logger.info(f"tokens: {' '.join([str(x) for x in tokens])}")
                    logger.info(f"input_ids: {' '.join([str(x) for x in input_ids])}")
                    logger.info(f"input_mask: {' '.join([str(x) for x in input_mask])}")
                    logger.info(f"segment_ids: {' '.join([str(x) for x in segment_ids])}")

                feature = InputFeature(input_ids = input_ids,
                                       input_mask = input_mask,
                                       segment_ids = segment_ids,
                                       label_id = label_id,
                                       input_len = input_len)
                features.append(feature)
                pbar.batch_step(step=ex_id, info={}, bar_type='create features')
            logger.info("Saving features into cached file %s", cached_features_file)
            torch.save(features, cached_features_file)
        return features

    def create_dataset(self,features,is_sorted = False):
        # Convert to Tensors and build dataset
        if is_sorted:
            logger.info("sorted data by th length of input")
            features = sorted(features,key=lambda x:x.input_len,reverse=True)
        all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
#        for i in features:
#            print(i.label_id)
        all_label_ids = torch.tensor([f.label_id for f in features],dtype=torch.long)
        dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
        return dataset

Overwriting pybert/io/bert_processor.py


In [0]:
%pycat pybert/io/bert_processor.py

In [0]:
 %%writefile run_bert.py 

import torch
import warnings
from pathlib import Path
from argparse import ArgumentParser
from pybert.train.losses import BCEWithLogLoss
from pybert.train.trainer import Trainer
from torch.utils.data import DataLoader
from pybert.io.bert_processor import BertProcessor
from pybert.common.tools import init_logger, logger
from pybert.common.tools import seed_everything
from pybert.configs.basic_config import config
from pybert.model.nn.bert_for_multi_label import BertForMultiLable
from pybert.preprocessing.preprocessor import EnglishPreProcessor
from pybert.callback.modelcheckpoint import ModelCheckpoint
from pybert.callback.trainingmonitor import TrainingMonitor
from pybert.train.metrics import AUC, AccuracyThresh, MultiLabelReport,Recall,Acc
from pytorch_transformers import AdamW, WarmupLinearSchedule
from torch.utils.data import RandomSampler, SequentialSampler

warnings.filterwarnings("ignore")


def run_train(args):
    # --------- data
    processor = BertProcessor(vocab_path=config['bert_vocab_path'], do_lower_case=args.do_lower_case)
    label_list = processor.get_labels()
    label2id = {label: i for i, label in enumerate(label_list)}
    id2label = {i: label for i, label in enumerate(label_list)}

    train_data = processor.get_train(config['data_dir'] / f"{args.data_name}.train.pkl")
    train_examples = processor.create_examples(lines=train_data,
                                               example_type='train',
                                               cached_examples_file=config[
                                                    'data_dir'] / f"cached_train_examples_{args.arch}")
    train_features = processor.create_features(examples=train_examples,
                                               max_seq_len=args.train_max_seq_len,
                                               cached_features_file=config[
                                                    'data_dir'] / "cached_train_features_{}_{}".format(
                                                   args.train_max_seq_len, args.arch
                                               ))
#    print(train_features[0].input_ids)
#    print(train_features[0].segment_ids)
#    print(train_features[0].label_id)
#    print(train_features[0].input_mask)
    train_dataset = processor.create_dataset(train_features, is_sorted=args.sorted)
    if args.sorted:
        train_sampler = SequentialSampler(train_dataset)
    else:
        train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

    valid_data = processor.get_dev(config['data_dir'] / f"{args.data_name}.valid.pkl")
    valid_examples = processor.create_examples(lines=valid_data,
                                               example_type='valid',
                                               cached_examples_file=config[
                                                                        'data_dir'] / f"cached_valid_examples_{args.arch}")

    valid_features = processor.create_features(examples=valid_examples,
                                               max_seq_len=args.eval_max_seq_len,
                                               cached_features_file=config[
                                                                        'data_dir'] / "cached_valid_features_{}_{}".format(
                                                   args.eval_max_seq_len, args.arch
                                               ))
    valid_dataset = processor.create_dataset(valid_features)
    valid_sampler = SequentialSampler(valid_dataset)
    valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=args.eval_batch_size)

    # ------- model
    logger.info("initializing model")
    if args.resume_path:
        args.resume_path = Path(args.resume_path)
        model = BertForMultiLable.from_pretrained(args.resume_path)
    else:
        model = BertForMultiLable.from_pretrained(config['bert_model_dir'])
    t_total = int(len(train_dataloader) / args.gradient_accumulation_steps * args.epochs)

    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],'weight_decay': args.weight_decay},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    warmup_steps = int(t_total * args.warmup_proportion)
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    lr_scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_steps, t_total=t_total)

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # ---- callbacks
    logger.info("initializing callbacks")
    train_monitor = TrainingMonitor(file_dir=config['figure_dir'], arch=args.arch)
    model_checkpoint = ModelCheckpoint(checkpoint_dir=config['checkpoint_dir'],mode=args.mode,
                                       monitor=args.monitor,arch=args.arch,
                                       save_best_only=args.save_best)

    # **************************** training model ***********************
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_examples))
    logger.info("  Num Epochs = %d", args.epochs)
    logger.info("  Total train batch size (w. parallel, distributed & accumulation) = %d",
                args.train_batch_size * args.gradient_accumulation_steps * (
                    torch.distributed.get_world_size() if args.local_rank != -1 else 1))
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    trainer = Trainer(n_gpu=args.n_gpu,
                      model=model,
                      epochs=args.epochs,
                      logger=logger,
                      criterion=BCEWithLogLoss(reduction='sum'),
                      optimizer=optimizer,
                      lr_scheduler=lr_scheduler,
                      early_stopping=None,
                      training_monitor=train_monitor,
                      fp16=args.fp16,
                      resume_path=args.resume_path,
                      grad_clip=args.grad_clip,
                      model_checkpoint=model_checkpoint,
                      gradient_accumulation_steps=args.gradient_accumulation_steps,
                      batch_metrics=[AccuracyThresh(thresh=0.5),Recall(),Acc()],
                      epoch_metrics=[#AUC(average='micro', task_type='binary'),
                                     #MultiLabelReport(id2label=id2label)
                                     AccuracyThresh(thresh=0.5),
                                     Recall(),Acc()])
    trainer.train(train_data=train_dataloader, valid_data=valid_dataloader, seed=args.seed)


def run_test(args):
    from pybert.io.task_data import TaskData
    from pybert.test.predictor import Predictor
    data = TaskData()
    targets, sentences = data.read_data(raw_data_path=config['test_path'],
                                        preprocessor=EnglishPreProcessor(),
                                        is_train=False)
    lines = list(zip(sentences, targets))
    processor = BertProcessor(vocab_path=config['bert_vocab_path'], do_lower_case=args.do_lower_case)
    label_list = processor.get_labels()
    id2label = {i: label for i, label in enumerate(label_list)}

    test_data = processor.get_test(lines=lines)
    test_examples = processor.create_examples(lines=test_data,
                                              example_type='test',
                                              cached_examples_file=config[
                                                                       'data_dir'] / f"cached_test_examples_{args.arch}")
    test_features = processor.create_features(examples=test_examples,
                                              max_seq_len=args.eval_max_seq_len,
                                              cached_features_file=config[
                                                                       'data_dir'] / "cached_test_features_{}_{}".format(
                                                  args.eval_max_seq_len, args.arch
                                              ))
    test_dataset = processor.create_dataset(test_features)
    test_sampler = SequentialSampler(test_dataset)
    test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=args.train_batch_size)
    model = BertForMultiLable.from_pretrained(config['checkpoint_dir'], num_labels=len(label_list))

    # ----------- predicting
    logger.info('model predicting....')
    predictor = Predictor(model=model,
                          logger=logger,
                          n_gpu=args.n_gpu)
    result = predictor.predict(data=test_dataloader)
    print(result)


def main():
    parser = ArgumentParser()
    parser.add_argument("--arch", default='bert', type=str)
    parser.add_argument("--do_data", action='store_true')
    parser.add_argument("--do_train", action='store_true')
    parser.add_argument("--do_test", action='store_true')
    parser.add_argument("--save_best", action='store_true')
    parser.add_argument("--do_lower_case", action='store_true')
    parser.add_argument('--data_name', default='kaggle', type=str)
    parser.add_argument("--epochs", default=6, type=int)
    parser.add_argument("--resume_path", default='', type=str)
    parser.add_argument("--mode", default='min', type=str)
    parser.add_argument("--monitor", default='valid_loss', type=str)
    parser.add_argument("--valid_size", default=0.2, type=float)
    parser.add_argument("--local_rank", type=int, default=-1)
    parser.add_argument("--sorted", default=1, type=int, help='1 : True  0:False ')
    parser.add_argument("--n_gpu", type=str, default='0', help='"0,1,.." or "0" or "" ')
    parser.add_argument('--gradient_accumulation_steps', type=int, default=1)
    parser.add_argument("--train_batch_size", default=8, type=int)
    parser.add_argument('--eval_batch_size', default=8, type=int)
    parser.add_argument("--train_max_seq_len", default=256, type=int)
    parser.add_argument("--eval_max_seq_len", default=256, type=int)
    parser.add_argument('--loss_scale', type=float, default=0)
    parser.add_argument("--warmup_proportion", default=0.1, type=int, )
    parser.add_argument("--weight_decay", default=0.01, type=float)
    parser.add_argument("--adam_epsilon", default=1e-8, type=float)
    parser.add_argument("--grad_clip", default=1.0, type=float)
    parser.add_argument("--learning_rate", default=2e-5, type=float)
    parser.add_argument('--seed', type=int, default=42)
    parser.add_argument('--fp16', action='store_true')
    parser.add_argument('--fp16_opt_level', type=str, default='O1')

    args = parser.parse_args()
    config['checkpoint_dir'] = config['checkpoint_dir'] / args.arch
    config['checkpoint_dir'].mkdir(exist_ok=True)
    # Good practice: save your training arguments together with the trained model
    torch.save(args, config['checkpoint_dir'] / 'training_args.bin')
    seed_everything(args.seed)
    init_logger(log_file=config['log_dir'] / f"{args.arch}.log")

    logger.info("Training/evaluation parameters %s", args)

    if args.do_data:
        from pybert.io.task_data import TaskData
        data = TaskData()
        targets, sentences = data.read_data(raw_data_path=config['raw_data_path'],
                                            preprocessor=EnglishPreProcessor(),
                                            is_train=True)
        data.train_val_split(X=sentences, y=targets, shuffle=True, stratify=False,
                             valid_size=args.valid_size, data_dir=config['data_dir'],
                             data_name=args.data_name)
    if args.do_train:
        run_train(args)

    if args.do_test:
        run_test(args)


if __name__ == '__main__':
    main()

Overwriting run_bert.py


In [0]:
import torch
import warnings
from pathlib import Path
from argparse import ArgumentParser
from pybert.train.losses import BCEWithLogLoss
from pybert.train.trainer import Trainer
from torch.utils.data import DataLoader
from pybert.io.bert_processor import BertProcessor
from pybert.common.tools import init_logger, logger
from pybert.common.tools import seed_everything
from pybert.configs.basic_config import config
from pybert.model.nn.bert_for_multi_label import BertForMultiLable
from pybert.preprocessing.preprocessor import EnglishPreProcessor
from pybert.callback.modelcheckpoint import ModelCheckpoint
from pybert.callback.trainingmonitor import TrainingMonitor
from pybert.train.metrics import AUC, AccuracyThresh, MultiLabelReport,Recall,Acc
from pytorch_transformers import AdamW, WarmupLinearSchedule
from torch.utils.data import RandomSampler, SequentialSampler

warnings.filterwarnings("ignore")


def run_train(args):
    # --------- data
    processor = BertProcessor(vocab_path=config['bert_vocab_path'], do_lower_case=args.do_lower_case)
    label_list = processor.get_labels()
    label2id = {label: i for i, label in enumerate(label_list)}
    id2label = {i: label for i, label in enumerate(label_list)}

    train_data = processor.get_train(config['data_dir'] / f"{args.data_name}.train.pkl")
    train_examples = processor.create_examples(lines=train_data,
                                               example_type='train',
                                               cached_examples_file=config[
                                                    'data_dir'] / f"cached_train_examples_{args.arch}")
    train_features = processor.create_features(examples=train_examples,
                                               max_seq_len=args.train_max_seq_len,
                                               cached_features_file=config[
                                                    'data_dir'] / "cached_train_features_{}_{}".format(
                                                   args.train_max_seq_len, args.arch
                                               ))
#    print(train_features[0].input_ids)
#    print(train_features[0].segment_ids)
#    print(train_features[0].label_id)
#    print(train_features[0].input_mask)
    train_dataset = processor.create_dataset(train_features, is_sorted=args.sorted)
    if args.sorted:
        train_sampler = SequentialSampler(train_dataset)
    else:
        train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

    valid_data = processor.get_dev(config['data_dir'] / f"{args.data_name}.valid.pkl")
    valid_examples = processor.create_examples(lines=valid_data,
                                               example_type='valid',
                                               cached_examples_file=config[
                                                                        'data_dir'] / f"cached_valid_examples_{args.arch}")

    valid_features = processor.create_features(examples=valid_examples,
                                               max_seq_len=args.eval_max_seq_len,
                                               cached_features_file=config[
                                                                        'data_dir'] / "cached_valid_features_{}_{}".format(
                                                   args.eval_max_seq_len, args.arch
                                               ))
    valid_dataset = processor.create_dataset(valid_features)
    valid_sampler = SequentialSampler(valid_dataset)
    valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=args.eval_batch_size)

    # ------- model
    logger.info("initializing model")
    if args.resume_path:
        args.resume_path = Path(args.resume_path)
        model = BertForMultiLable.from_pretrained(args.resume_path)
    else:
        model = BertForMultiLable.from_pretrained(config['bert_model_dir'])
    t_total = int(len(train_dataloader) / args.gradient_accumulation_steps * args.epochs)

    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],'weight_decay': args.weight_decay},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    warmup_steps = int(t_total * args.warmup_proportion)
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    lr_scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_steps, t_total=t_total)

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # ---- callbacks
    logger.info("initializing callbacks")
    train_monitor = TrainingMonitor(file_dir=config['figure_dir'], arch=args.arch)
    model_checkpoint = ModelCheckpoint(checkpoint_dir=config['checkpoint_dir'],mode=args.mode,
                                       monitor=args.monitor,arch=args.arch,
                                       save_best_only=args.save_best)

    # **************************** training model ***********************
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_examples))
    logger.info("  Num Epochs = %d", args.epochs)
    logger.info("  Total train batch size (w. parallel, distributed & accumulation) = %d",
                args.train_batch_size * args.gradient_accumulation_steps * (
                    torch.distributed.get_world_size() if args.local_rank != -1 else 1))
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    trainer = Trainer(n_gpu=args.n_gpu,
                      model=model,
                      epochs=args.epochs,
                      logger=logger,
                      criterion=BCEWithLogLoss(),
                      optimizer=optimizer,
                      lr_scheduler=lr_scheduler,
                      early_stopping=None,
                      training_monitor=train_monitor,
                      fp16=args.fp16,
                      resume_path=args.resume_path,
                      grad_clip=args.grad_clip,
                      model_checkpoint=model_checkpoint,
                      gradient_accumulation_steps=args.gradient_accumulation_steps,
                      batch_metrics=[AccuracyThresh(thresh=0.5),Recall(),Acc()],
                      epoch_metrics=[#AUC(average='micro', task_type='binary'),
                                     #MultiLabelReport(id2label=id2label)
                                     AccuracyThresh(thresh=0.5),
                                     Recall(),Acc()])
    trainer.train(train_data=train_dataloader, valid_data=valid_dataloader, seed=args.seed)


def run_test(args):
    from pybert.io.task_data import TaskData
    from pybert.test.predictor import Predictor
    data = TaskData()
    targets, sentences = data.read_data(raw_data_path=config['test_path'],
                                        preprocessor=EnglishPreProcessor(),
                                        is_train=False)
    lines = list(zip(sentences, targets))
    processor = BertProcessor(vocab_path=config['bert_vocab_path'], do_lower_case=args.do_lower_case)
    label_list = processor.get_labels()
    id2label = {i: label for i, label in enumerate(label_list)}

    test_data = processor.get_test(lines=lines)
    test_examples = processor.create_examples(lines=test_data,
                                              example_type='test',
                                              cached_examples_file=config[
                                                                       'data_dir'] / f"cached_test_examples_{args.arch}")
    test_features = processor.create_features(examples=test_examples,
                                              max_seq_len=args.eval_max_seq_len,
                                              cached_features_file=config[
                                                                       'data_dir'] / "cached_test_features_{}_{}".format(
                                                  args.eval_max_seq_len, args.arch
                                              ))
    test_dataset = processor.create_dataset(test_features)
    test_sampler = SequentialSampler(test_dataset)
    test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=args.train_batch_size)
    model = BertForMultiLable.from_pretrained(config['checkpoint_dir'], num_labels=len(label_list))

    # ----------- predicting
    logger.info('model predicting....')
    predictor = Predictor(model=model,
                          logger=logger,
                          n_gpu=args.n_gpu)
    result = predictor.predict(data=test_dataloader)
    print(result)


def main():
    parser = ArgumentParser()
    parser.add_argument("--arch", default='bert', type=str)
    parser.add_argument("--do_data", action='store_true')
    parser.add_argument("--do_train", action='store_true')
    parser.add_argument("--do_test", action='store_true')
    parser.add_argument("--save_best", action='store_true')
    parser.add_argument("--do_lower_case", action='store_true')
    parser.add_argument('--data_name', default='kaggle', type=str)
    parser.add_argument("--epochs", default=6, type=int)
    parser.add_argument("--resume_path", default='', type=str)
    parser.add_argument("--mode", default='min', type=str)
    parser.add_argument("--monitor", default='valid_loss', type=str)
    parser.add_argument("--valid_size", default=0.2, type=float)
    parser.add_argument("--local_rank", type=int, default=-1)
    parser.add_argument("--sorted", default=1, type=int, help='1 : True  0:False ')
    parser.add_argument("--n_gpu", type=str, default='0', help='"0,1,.." or "0" or "" ')
    parser.add_argument('--gradient_accumulation_steps', type=int, default=1)
    parser.add_argument("--train_batch_size", default=8, type=int)
    parser.add_argument('--eval_batch_size', default=8, type=int)
    parser.add_argument("--train_max_seq_len", default=256, type=int)
    parser.add_argument("--eval_max_seq_len", default=256, type=int)
    parser.add_argument('--loss_scale', type=float, default=0)
    parser.add_argument("--warmup_proportion", default=0.1, type=int, )
    parser.add_argument("--weight_decay", default=0.01, type=float)
    parser.add_argument("--adam_epsilon", default=1e-8, type=float)
    parser.add_argument("--grad_clip", default=1.0, type=float)
    parser.add_argument("--learning_rate", default=2e-5, type=float)
    parser.add_argument('--seed', type=int, default=42)
    parser.add_argument('--fp16', action='store_true')
    parser.add_argument('--fp16_opt_level', type=str, default='O1')

    args = parser.parse_args()
    config['checkpoint_dir'] = config['checkpoint_dir'] / args.arch
    config['checkpoint_dir'].mkdir(exist_ok=True)
    # Good practice: save your training arguments together with the trained model
    torch.save(args, config['checkpoint_dir'] / 'training_args.bin')
    seed_everything(args.seed)
    init_logger(log_file=config['log_dir'] / f"{args.arch}.log")

    logger.info("Training/evaluation parameters %s", args)

    if args.do_data:
        from pybert.io.task_data import TaskData
        data = TaskData()
        targets, sentences = data.read_data(raw_data_path=config['raw_data_path'],
                                            preprocessor=EnglishPreProcessor(),
                                            is_train=True)
        data.train_val_split(X=sentences, y=targets, shuffle=True, stratify=False,
                             valid_size=args.valid_size, data_dir=config['data_dir'],
                             data_name=args.data_name)
    if args.do_train:
        run_train(args)

    if args.do_test:
        run_test(args)


if __name__ == '__main__':
    main()

usage: ipykernel_launcher.py [-h] [--arch ARCH] [--do_data] [--do_train]
                             [--do_test] [--save_best] [--do_lower_case]
                             [--data_name DATA_NAME] [--epochs EPOCHS]
                             [--resume_path RESUME_PATH] [--mode MODE]
                             [--monitor MONITOR] [--valid_size VALID_SIZE]
                             [--local_rank LOCAL_RANK] [--sorted SORTED]
                             [--n_gpu N_GPU]
                             [--gradient_accumulation_steps GRADIENT_ACCUMULATION_STEPS]
                             [--train_batch_size TRAIN_BATCH_SIZE]
                             [--eval_batch_size EVAL_BATCH_SIZE]
                             [--train_max_seq_len TRAIN_MAX_SEQ_LEN]
                             [--eval_max_seq_len EVAL_MAX_SEQ_LEN]
                             [--loss_scale LOSS_SCALE]
                             [--warmup_proportion WARMUP_PROPORTION]
                             [--weight

SystemExit: ignored

In [0]:
%tb

SystemExit: ignored

In [0]:
import numpy as np
import pandas as pd
import os

import keras
from collections import defaultdict

Using TensorFlow backend.
/home/iimtech8/.conda/envs/matplotlib/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/iimtech8/.conda/envs/matplotlib/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/iimtech8/.conda/envs/matplotlib/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/iimte

In [0]:
raw_data_path = '/home/iimtech8/Desktop/Bert-Multi-Label-Text-Classification-master/pybert/dataset/train.csv'

In [0]:
data = pd.read_hdf(raw_data_path)		
for row in data.values:
        target = row[0]

In [0]:
target

In [0]:

import pytorch_transformers

In [0]:
config = pytorch_transformers.BertConfig('/home/iimtech8/Desktop/Bert-Multi-Label-Text-Classification-master/pybert/pretrain/bert/base-uncased/config.json')

In [0]:
x = pytorch_transformers.BertPreTrainedModel(config)

In [0]:
help(x.apply)

Help on method apply in module torch.nn.modules.module:

apply(fn) method of pytorch_transformers.modeling_bert.BertPreTrainedModel instance
    Applies ``fn`` recursively to every submodule (as returned by ``.children()``)
    as well as self. Typical use includes initializing the parameters of a model
    (see also :ref:`torch-nn-init`).
    
    Args:
        fn (:class:`Module` -> None): function to be applied to each submodule
    
    Returns:
        Module: self
    
    Example::
    
        >>> def init_weights(m):
        >>>     print(m)
        >>>     if type(m) == nn.Linear:
        >>>         m.weight.data.fill_(1.0)
        >>>         print(m.weight)
        >>> net = nn.Sequential(nn.Linear(2, 2), nn.Linear(2, 2))
        >>> net.apply(init_weights)
        Linear(in_features=2, out_features=2, bias=True)
        Parameter containing:
        tensor([[ 1.,  1.],
                [ 1.,  1.]])
        Linear(in_features=2, out_features=2, bias=True)
        Parameter 

In [0]:
help(x.init_weights)

Help on method init_weights in module pytorch_transformers.modeling_utils:

init_weights() method of pytorch_transformers.modeling_bert.BertPreTrainedModel instance
    Initialize and prunes weights if needed.



In [0]:
import torch.nn as nn

In [0]:
y = nn.Module()

In [0]:
config

{
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 200,
  "output_attentions": false,
  "output_hidden_states": false,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

In [0]:
print(x.init_weights())

None


In [0]:
from pybert.metric import AccuracyThresh

metric = AccuracyThresh()
for epoch in range(epochs):
    metric.reset()
    for batch in batchs:
        logits = model()
        metric(logits,target)
        print(metric.name(),metric.value())

ModuleNotFoundError: No module named 'pybert.metric'

In [0]:
df = pd.read_hdf('pybert/dataset/train.csv')

In [0]:
df

,sentence,label,med_vec
3000,family history nan. past history nan. clinical...,"[39, 9, 51]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3001,family history nan. past history nan. clinical...,"[13, 4, 33, 45]","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
3002,family history nan. past history nan. clinical...,"[4, 51]","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
3003,family history nan. past history nan. clinical...,"[58, 9]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3004,family history nan. past history nan. clinical...,"[4, 72]","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...
15551,family history nan. past history nan. clinical...,"[1, 8, 149, 37]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
15552,family history nan. past history nan. clinical...,"[37, 68, 72]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
15553,family history nan. past history nan. clinical...,"[73, 8]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
15554,family history nan. past history nan. clinical...,"[8, 1, 51]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."


In [0]:
df.columns = ['sentence', 'label']

# Create sentence and label lists
sentences = df.sentence.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = df.label

df



sentences



## Inputs

labels

import keras
NUM_MEDICINES = 200
tokenizer = keras.preprocessing.text.Tokenizer(num_words=NUM_MEDICINES)


tokenizer.sequences_to_matrix([[1,2,3]])

labels = df['label'].apply(lambda x:[x])

df.label[1]

# df['medicines_y'] = df['label'].apply(lambda x:[x])

df['med_vec'] = labels.apply(tokenizer.sequences_to_matrix)

df.med_vec

0        [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...
1        [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...
2        [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
3        [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...
4        [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...
5        [[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0,...
6        [[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0,...
7        [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
8        [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...
9        [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...
10       [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
11       [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
12       [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
13       [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...
14       [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...
15       [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
16       [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,.

In [0]:
df.med_vec = df.med_vec.apply(lambda x:x[0])

In [0]:
x = df.med_vec.iloc[:100]

In [0]:
x.to_hdf('small_bert_input.h5','df')

/home/sai/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:1993: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->values] [items->None]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [0]:
df.to_hdf('uc_bert_input.h5','df')

/home/sai/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:1993: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['sentence', 'label', 'med_vec']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [0]:
import pandas as pd
df = pd.read_hdf('pybert/dataset/test.csv')

In [0]:
df

,sentence,label,med_vec
0,family history nan. past history NIL. clinical...,"[12, 6, 12, 6, 12, 9, 12]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
1,family history No Family history of DM. past h...,"[13, 6]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
2,family history No Family History. past history...,[14],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,family history Test. past history Test. clinic...,"[6, 10, 35, 17]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
4,family history Father Diabetic. past history D...,"[10, 7]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
...,...,...,...
95,family history nan. past history h/o surgery f...,"[4, 48]","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
96,family history nan. past history prev brest lu...,[48],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
97,family history nan. past history UNDERGONE ANG...,"[8, 43]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
98,family history nan. past history UNDERGONE ANG...,"[67, 8, 51]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."


In [0]:
for rows in df.values:
  target = rows[1]
  sentence = rows[0]

In [0]:
sentence
# target

'family history nan. past history Diabetic,. clinical_findings nan. clinical_notes nan. decision nan. presenting_complaints body pain,. final_diagnosis nan.'

In [0]:
%pycat pybert/train/trainer.py

In [0]:
%pycat  pybert/model/nn/bert_for_multi_label.py


In [0]:
import numpy as np